In [1]:
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
import keras.backend as K
from keras.optimizers import Adam
from keras.models import Sequential
from keras.utils import Sequence
from keras.layers import *
import matplotlib.pyplot as plt

In [2]:
path = "input/"
data = pd.read_csv(path+"sudoku.csv")
try:
    data = pd.DataFrame({"quizzes":data["puzzle"],"solutions":data["solution"]})
except:
    pass

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 2 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   quizzes    1000000 non-null  object
 1   solutions  1000000 non-null  object
dtypes: object(2)
memory usage: 15.3+ MB


In [3]:
data.head()

,quizzes,solutions
0,0043002090050090010700600430060020871900074000...,8643712593258497619712658434361925871986574322...
1,0401000501070039605200080000000000170009068008...,3461792581875239645296483719658324174729168358...
2,6001203840084590720000060050002640300700800069...,6951273841384596727248369158512647392739815469...
3,4972000001004000050000160986203000403009000000...,4972583161864397252537164986293815473759641828...
4,0059103080094030600275001000300002010008200070...,4659123781894735623275681497386452919548216372...


In [4]:
print("Quiz:\n",np.array(list(map(int,list(data['quizzes'][0])))).reshape(9,9))
print("Solution:\n",np.array(list(map(int,list(data['solutions'][0])))).reshape(9,9))


Quiz:
 [[0 0 4 3 0 0 2 0 9]
 [0 0 5 0 0 9 0 0 1]
 [0 7 0 0 6 0 0 4 3]
 [0 0 6 0 0 2 0 8 7]
 [1 9 0 0 0 7 4 0 0]
 [0 5 0 0 8 3 0 0 0]
 [6 0 0 0 0 0 1 0 5]
 [0 0 3 5 0 8 6 9 0]
 [0 4 2 9 1 0 3 0 0]]
Solution:
 [[8 6 4 3 7 1 2 5 9]
 [3 2 5 8 4 9 7 6 1]
 [9 7 1 2 6 5 8 4 3]
 [4 3 6 1 9 2 5 8 7]
 [1 9 8 6 5 7 4 3 2]
 [2 5 7 4 8 3 9 1 6]
 [6 8 9 7 3 4 1 2 5]
 [7 1 3 5 2 8 6 9 4]
 [5 4 2 9 1 6 3 7 8]]


In [4]:
class DataGenerator(Sequence):
    def __init__(self, df,batch_size = 16,subset = "train",shuffle = False, info={}):
        super().__init__()
        self.df = df
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.subset = subset
        self.info = info
        
        self.data_path = path
        self.on_epoch_end()
        
    def __len__(self):
        return int(np.floor(len(self.df)/self.batch_size))
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.df))
        if self.shuffle==True:
            np.random.shuffle(self.indexes)
            
    def __getitem__(self,index):
        X = np.empty((self.batch_size, 9,9,1))
        y = np.empty((self.batch_size,81,1))
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        for i,f in enumerate(self.df['quizzes'].iloc[indexes]):
            self.info[index*self.batch_size+i]=f
            X[i,] = (np.array(list(map(int,list(f)))).reshape((9,9,1))/9)-0.5
        if self.subset == 'train': 
            for i,f in enumerate(self.df['solutions'].iloc[indexes]):
                self.info[index*self.batch_size+i]=f
                y[i,] = np.array(list(map(int,list(f)))).reshape((81,1)) - 1
        if self.subset == 'train': return X, y
        else: return X

In [5]:
train_idx = int(len(data)*0.95)
data = data.sample(frac=1).reset_index(drop=True)
training_generator = DataGenerator(data.iloc[:train_idx], subset = "train", batch_size=320)
validation_generator = DataGenerator(data.iloc[train_idx:], subset = "train",  batch_size=320)        

In [7]:
type(training_generator)

__main__.DataGenerator

In [6]:
training_generator.__getitem__(4)[0].shape

(320, 9, 9, 1)

In [7]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3,3), activation='relu', padding='same', input_shape=(9,9,1)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=(1,1), activation='relu', padding='same'))

model.add(Flatten())
model.add(Dense(81*9))
model.add(Reshape((-1, 9)))
model.add(Activation('softmax'))

adam = tf.keras.optimizers.Adam(lr=.001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 9, 9, 32)          320       
_________________________________________________________________
batch_normalization_2 (Batch (None, 9, 9, 32)          128       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 9, 9, 64)          18496     
_________________________________________________________________
batch_normalization_3 (Batch (None, 9, 9, 64)          256       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 9, 9, 128)         8320      
_________________________________________________________________
flatten_1 (Flatten)          (None, 10368)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 729)              

In [8]:
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau
filepath1="weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
filepath2 = "best_weights.hdf5"
checkpoint1 = ModelCheckpoint(filepath1, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
checkpoint2 = ModelCheckpoint(filepath2, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    patience=3,
    verbose=1,
    min_lr=1e-6
)
callbacks_list = [checkpoint1,checkpoint2,reduce_lr]

In [9]:
history = model.fit_generator(training_generator, validation_data = validation_generator, epochs = 3, verbose=1,callbacks=callbacks_list )

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/3
2968/2968 [==============================] - ETA: 0s - loss: 0.6035 - accuracy: 0.1123
Epoch 00001: val_accuracy improved from -inf to 0.11330, saving model to weights-improvement-01-0.11.hdf5

Epoch 00001: val_accuracy improved from -inf to 0.11330, saving model to best_weights.hdf5
2968/2968 [==============================] - 995s 335ms/step - loss: 0.6035 - accuracy: 0.1123 - val_loss: 0.3952 - val_accuracy: 0.1133
Epoch 2/3
2968/2968 [==============================] - ETA: 0s - loss: 0.3784 - accuracy: 0.1112
Epoch 00002: val_accuracy did not improve from 0.11330

Epoch 00002: val_accuracy did not improve from 0.11330
2968/2968 [==============================] - 1002s 338ms/step - loss: 0.3784 - accuracy: 0.1112 - val_loss: 0.3698 - val_accuracy: 0.1092
Epoch 3/3
2968/2968 [==============================] - ETA: 0s - loss: 0.3594 - accuracy: 0.1112
Epoch 00003: val_accuracy did not improve from 0.

In [10]:
model.load_weights('best_weights.hdf5')

In [11]:
def norm(a):
    return (a/9)-.5

def denorm(a):
    return (a+.5)*9

def inference_sudoku(sample):
    
    '''
        This function solve the sudoku by filling blank positions one by one.
    '''
    
    feat = sample
    
    while(1):
    
        out = model.predict(feat.reshape((1,9,9,1)))  
        out = out.squeeze()

        pred = np.argmax(out, axis=1).reshape((9,9))+1 
        prob = np.around(np.max(out, axis=1).reshape((9,9)), 2) 
        
        feat = denorm(feat).reshape((9,9))
        mask = (feat==0)
     
        if(mask.sum()==0):
            break
            
        prob_new = prob*mask
    
        ind = np.argmax(prob_new)
        x, y = (ind//9), (ind%9)

        val = pred[x][y]
        feat[x][y] = val
        feat = norm(feat)
    
    return pred

def test_accuracy(feats, labels):
    
    correct = 0
    
    for i,feat in enumerate(feats):
        
        pred = inference_sudoku(feat)
        
        true = labels[i].reshape((9,9))+1
        
        if(abs(true - pred).sum()==0):
            correct += 1
        
    print(correct/feats.shape[0])

def solve_sudoku(game):
    try:
        game = game.replace('\n', '')
        game = game.replace(' ', '')
        game = np.array([int(j) for j in game]).reshape((9,9,1))   
    except: 
        pass
   
    game = game.reshape((9,9,1))
    print(game)
    game = norm(game)
    game = inference_sudoku(game)
    return game

In [ ]:
game = '''
          0 0 0 7 0 0 0 9 6
          0 0 3 0 6 9 1 7 8
          0 0 7 2 0 0 5 0 0
          0 7 5 0 0 0 0 0 0
          9 0 1 0 0 0 3 0 0
          0 0 0 0 0 0 0 0 0
          0 0 9 0 0 0 0 0 1
          3 1 8 0 2 0 4 0 7
          2 4 0 0 0 5 0 0 0
      '''

In [12]:

game = solve_sudoku(game)

print('solved puzzle:\n')
print(game)

NameError: name 'game' is not defined

In [13]:
entree_facile = np.array([[9, 0, 6, 4, 0, 5, 0, 8, 0],
                 [0, 7, 2, 0, 0, 0, 0, 4, 1],
                 [3, 4, 0, 0, 0, 2, 0, 0, 0],
                 [0, 0, 4, 1, 6, 0, 3, 0, 0],
                 [7, 0, 1, 0, 4, 0, 9, 0, 2],
                 [0, 0, 5, 0, 3, 7, 4, 0, 0],
                 [0, 0, 0, 7, 0, 0, 0, 2, 9],
                 [8, 2, 0, 0, 0, 0, 1, 7, 0],
                 [0, 5, 0, 9, 0, 6, 8, 0, 4]])
game = solve_sudoku(entree_facile)


[[[9]
  [0]
  [6]
  [4]
  [0]
  [5]
  [0]
  [8]
  [0]]

 [[0]
  [7]
  [2]
  [0]
  [0]
  [0]
  [0]
  [4]
  [1]]

 [[3]
  [4]
  [0]
  [0]
  [0]
  [2]
  [0]
  [0]
  [0]]

 [[0]
  [0]
  [4]
  [1]
  [6]
  [0]
  [3]
  [0]
  [0]]

 [[7]
  [0]
  [1]
  [0]
  [4]
  [0]
  [9]
  [0]
  [2]]

 [[0]
  [0]
  [5]
  [0]
  [3]
  [7]
  [4]
  [0]
  [0]]

 [[0]
  [0]
  [0]
  [7]
  [0]
  [0]
  [0]
  [2]
  [9]]

 [[8]
  [2]
  [0]
  [0]
  [0]
  [0]
  [1]
  [7]
  [0]]

 [[0]
  [5]
  [0]
  [9]
  [0]
  [6]
  [8]
  [0]
  [4]]]


In [14]:
print('solved puzzle:\n')
print(game)

solved puzzle:

[[9 1 6 4 7 5 2 8 3]
 [5 7 2 8 9 3 6 4 1]
 [3 4 8 6 1 2 7 9 5]
 [2 8 4 1 6 9 3 5 7]
 [7 3 1 5 4 8 9 6 2]
 [6 9 5 2 3 7 4 1 8]
 [4 6 3 7 8 1 5 2 9]
 [8 2 9 3 5 4 1 7 6]
 [1 5 7 9 2 6 8 3 4]]
